In [1]:
# Import libraries
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer, inferno
from bokeh.models.annotations import Title
from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, WidgetBox, row, column
from bokeh.plotting import figure


from bokeh.palettes import Category20_16, inferno

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models import Label
from bokeh.events import DoubleTap,Tap
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from bokeh.models import (CategoricalColorMapper, HoverTool, 
                          ColumnDataSource, Panel, 
                          FuncTickFormatter, SingleIntervalTicker, LinearAxis)
from bokeh.models.widgets import (CheckboxGroup, Slider, RangeSlider, 
                                  Tabs, CheckboxButtonGroup, 
                                  TableColumn, DataTable, Select)


In [47]:
def map_tab(doc):
    df = pd.read_csv('../data/Properties_clean.csv')

    # Fill house square foot zero values with the average house square footage by bedroom
    average_data = df.groupby('BEDRM').AREA.mean()
    # Use average AREA by bedroom for each 0 value in each bedroom group, up to 14 bedrooms 
    for i in range(0, 14): 
        df.loc[(df['AREA'] == 0) & (df['BEDRM'] == i), 'AREA'] = average_data.loc[i]

    df['price_sf'] = df['PRICE'] / df['AREA']
    df = df[df['ASSESSMENT_NBHD'].notna()]
    df = df[df['PRICE'].notna()]
    df = df[df['AREA'].notna()]
    df = df[df['price_sf'].notna()]
    df = df[df['CENSUS_TRACT'].notna()]
    df = df[df['TSEYB'].notna()]
    df.TSEYB = df.TSEYB//10
    neighborhood_data = df.groupby(
        ['ASSESSMENT_NBHD', 'CENSUS_TRACT', 'TSEYB']
    ).agg(
        {
            'PRICE': ['count', 'mean', 'median'],
            'AREA': ['mean'],
            'price_sf': ['mean']
        }
    )
    
    #Reset the index to 1 level to fill in year
    neighborhood_data = neighborhood_data.set_axis(neighborhood_data.columns.map('_'.join), axis=1, inplace=False)
    neighborhood_data = neighborhood_data.reset_index(level=[0,1,2])

    # Change data types to integer 
    neighborhood_data = neighborhood_data.astype({'PRICE_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'PRICE_median': 'int'})
    neighborhood_data = neighborhood_data.astype({'AREA_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'price_sf_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'CENSUS_TRACT': 'int'})
    neighborhood_data = neighborhood_data.astype({'TSEYB': 'int'})
    

    # Read in shapefile and examine data
    dc = gpd.read_file('../data/Census_Tracts_in_2010.shp')
    # Set the Coordinate Referance System (crs) for projections
    # ESPG code 4326 is also referred to as WGS84 lat-long projection
    dc.crs = {'init': 'epsg:4326'}

    # Rename columns in geojson map file
    dc = dc.rename(columns={'geometry': 'geometry'}).set_geometry('geometry')


    dc.sort_values(by=['TRACT'])

    neighborhood_data.index = neighborhood_data.index.astype(int)
    dc.TRACT = dc.TRACT.astype(int)

    # This dictionary contains the formatting for the data in the plots
    format_data = [('PRICE_count', 0, 700,'0,0', 'Number of properties'),
                ('PRICE_mean', 0, 1_500_000,'$0,0', 'Average Sales Price'),
                ('PRICE_median', 0, 1_500_000, '$0,0', 'Median Sale Price'),
                ('AREA_mean', 500, 5000,'0,0', 'Average Square Footage'),
                ('price_sf_mean', 0, 1500,'$0,0', 'Mean Price Per SF')]
    
    #Create a DataFrame object from the dictionary 
    format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])
    def json_data(residential_type):
        tseyb = 0
        for key, value in use_code_dict.items():
            if residential_type == value:
                tseyb = key
                
        neighborhood_data_temp = neighborhood_data.loc[neighborhood_data.TSEYB == tseyb,:]
            
        # Merge the GeoDataframe object (dc) with the neighborhood summary data (neighborhood)
        merged = pd.merge(dc, neighborhood_data_temp, left_on='TRACT', right_on='CENSUS_TRACT', how='left')
        merged = merged.dropna()
        # Fill the null values
        values = {'PRICE_count': 0, 'PRICE_mean': 0, 'PRICE_median': 0, 'TSEYB':0,
                    'sf_mean': 0, 'price_sf_mean': 0, 'ASSESSMENT_NBHD': "", 'AREA_mean': 0}
        merged = merged.fillna(value=values)

        # Bokeh uses geojson formatting, representing geographical features, with json
        # Convert to json
        merged_json = json.loads(merged.to_json())
        # Convert to json preferred string-like object 
        json_data = json.dumps(merged_json)

        return json_data
    
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    def make_data(field_name):
        # Set the format of the colorbar
        min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
        max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
        field_format = 'price_sf_mean'# format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

        # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
        color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

        # Create color bar.
        format_tick = NumeralTickFormatter(format="$0.0")
        color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick,
        border_line_color=None, location = (0, 0))

        # Create figure object.
        verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]
        return field_name,verbage,color_bar
    
    def insert_data(p,field_name,verbage):
        t = Title()
        t.text =  verbage + ' by years since built'
        p.title = t

        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        p.axis.visible = False
        color_map_instance = p.select_one(LinearColorMapper)
        # Add patch renderer to figure. 
        p.patches('xs','ys', source = geosource, 
                  fill_color = {'field' : field_name, 'transform' : color_map_instance},
                  line_color = 'black', line_width = 0.25, fill_alpha = 1)

        # Add the hover tool to the graph
        p.add_tools(hover)
        return p
    
    # Create a plotting function
    def make_plot(field_name):    
        field_name,verbage,color_bar = make_data(field_name)
        
        
        p = figure(title = verbage + ' by Neighborhood for Homes in DC', 
                    plot_height = 700, plot_width = 600,
                    toolbar_location = None)
        
        p.add_layout(color_bar, 'right')
        p = insert_data(p,field_name,verbage)
        
        return p
    
    # Define the callback function: update_plot for updating the colormap
    def update_plot(attr, old, new):
        # The input cr is the criteria selected from the select box
        cr = select.value

        new_data = json_data(cr)

        
        input_field = 'price_sf_mean' #format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
        # Update the plot based on the changed inputs
        field_name,verbage,_ = make_data(input_field)

        insert_data(p,field_name,verbage)
        
        min_range = format_df.loc[format_df['field'] == input_field, 'min_range'].iloc[0]
        max_range = format_df.loc[format_df['field'] == input_field, 'max_range'].iloc[0]
        color_map_instance = p.select_one(LinearColorMapper)
        color_map_instance.update(low=min_range,high=max_range)

        # Update the data

        geosource.geojson = new_data
        
    def histogram_plot():
        # Create the blank plot
        p_temp = figure(plot_height = 600, plot_width = 400, 
                   title = 'Real Estate prices',
                   x_axis_label = 'Price ($)', 
                   y_axis_label = 'Number of properties')

        # Add a quad glyph with source this time
        p_temp.quad(bottom=0, top='property_count', left='left', right='right', source=src,
               fill_color='red', line_color='black', fill_alpha = 0.75,
               hover_fill_alpha = 1.0, hover_fill_color = 'navy')

        # Hover tool referring to our own data field using @ 
        h1 = HoverTool(tooltips = [('Number Of Properties', '@property_count'),
                                  ('Price Interval', '@p_interval')])

        p_temp.xaxis[0].formatter = NumeralTickFormatter(format="$0")

        # style the plot
        p_temp = style(p_temp)

        # add the hovertool
        p_temp.add_tools(h1)
        return p_temp
    
    def get_polygon_by_click(x, y):
        for i, polygon in enumerate(dc.geometry):
            if polygon.contains(Point(x, y)):
                clicked_tract = dc.TRACT.iloc[i]
                update_histogram(clicked_tract)

    def histogram_data(tract):
        df_temp = df.loc[df.CENSUS_TRACT == tract]
        ### for histogram
        arr_hist, edges = np.histogram(df_temp['PRICE'],
                                       bins = np.arange(0,1_500_000,50_000),
                                       range=[0,df_temp.PRICE.quantile(0.9)])
        prices = pd.DataFrame({'property_count': arr_hist, 
                               'left': edges[:-1], 
                               'right': edges[1:]})
        # Add a column showing the extent of each interval
        prices['p_interval'] = ['$%d to $%d' % (left, right) for left, right in zip(prices['left'], prices['right'])]        
        # Convert dataframe to column data source
        ### end for histogram
        return ColumnDataSource(prices)
        
                    
    def update_histogram(tract):
        new_data = histogram_data(tract)
        src.data.update(new_data.data)
        
    # add a dot where the click happened on map
    def callback(event):
        Coords=(event.x,event.y)
        get_polygon_by_click(event.x, event.y)
        coordList.append(Coords) 
        
    ### initial histogram setup 
    arr_hist, edges = np.histogram(df['PRICE'],
                                   bins = np.arange(0,1_500_000,50_000),
                                   range=[0,df.PRICE.quantile(0.75)])
    # Put the information in a dataframe
    prices = pd.DataFrame({'property_count': arr_hist, 
                           'left': edges[:-1], 
                           'right': edges[1:]})
    # Add a column showing the extent of each interval
    prices['p_interval'] = ['$%d to $%d' % (left, right) for left, right in zip(prices['left'], prices['right'])]        
    # Convert dataframe to column data source
    src = ColumnDataSource(prices)
    p_histogram = histogram_plot()
    p_histogram = style(p_histogram)
    p_histogram.xaxis.major_label_orientation = math.pi/4
    
    ### end for histogram
    
    
    ### shapefile census tract setup
    use_code_dict = {0: '0-9 years old',
                    1: '10-19  years old',
                    2: '20-29  years old',
                    3: '30-39  years old',
                    4: '40-49  years old',
                    5: '50-59  years old',
                    6: '60+ years old'}
    
    # Input geojson source that contains features for plotting 
    geosource = GeoJSONDataSource(geojson = json_data('0-4 years since built'))
    input_field = 'price_sf_mean'

    # Define a sequential multi-hue color palette.
    palette = inferno(100)

    # Reverse color order so that dark blue is highest obesity.
    #palette = palette[::-1]

        # Add hover tool
    hover = HoverTool(tooltips = [ ('Neighbourhood','@ASSESSMENT_NBHD'),
                                    ('Number of properties', '@PRICE_count'),
                                    ('Average Price', '$@PRICE_mean{,}'),
                                    ('Median Price', '$@PRICE_median{,}'),
                                    ('Average AREA', '@AREA_mean{,}'),
                                    ('Price/SF ', '$@price_sf_mean{,}')])

    
    # Call the plotting function
    p = make_plot(input_field)
    p = style(p)
    coordList=[]
    p.on_event(Tap, callback)
            
    select = Select(title='Select years since built:', value='Residential-Single Family', options=list(use_code_dict.values()))
    
    select.on_change('value', update_plot)
    ### shapefile census tract setup end
    # set up layout
    widgets = column(widgetbox(select))
    main_row = row(p, p_histogram)
    
    # Make a column layout of widgetbox and plot, and add it to the current document
    layout = column(main_row, widgets)

    doc.add_root(layout)

output_notebook()
show(map_tab)

Loading BokehJS ...

In [44]:
    df = pd.read_csv('../data/Properties_clean.csv')

    neighborhood_data = df.groupby(
        ['CENSUS_TRACT']
    ).agg(
        {
        'PRICE': ['count', 'median'],
        'AREA': ['mean'],
        'ASSESSMENT_NBHD': (lambda x: x.value_counts().index[0] if len(np.unique(x)) > 0 else "")
        }
    )
    
    neighborhood_data = neighborhood_data.set_axis(neighborhood_data.columns.map('_'.join), axis=1, inplace=False)
    neighborhood_data = neighborhood_data.rename({'ASSESSMENT_NBHD_<lambda>':'ASSESSMENT_NBHD'},axis='columns')

In [45]:
neighborhood_data

,PRICE_count,PRICE_median,AREA_mean,ASSESSMENT_NBHD
CENSUS_TRACT,,,,
0.0,11961,0.0,0.000000,0
100.0,1166,842250.0,3656.728787,Georgetown
202.0,909,827500.0,3351.950424,Georgetown
300.0,1182,670000.0,2825.216046,Glover Park
400.0,297,760000.0,5373.645914,Observatory Circle
...,...,...,...,...
10700.0,260,369950.0,1875.692308,Central-tri 1
10800.0,224,198000.0,1130.720000,Foggy Bottom
10900.0,83,75000.0,2139.163399,Congress Heights


In [34]:
    #Reset the index to 1 level to fill in year
    neighborhood_data = neighborhood_data.set_axis(neighborhood_data.columns.map('_'.join), axis=1, inplace=False)
    neighborhood_data = neighborhood_data.reset_index(level=[0])

    # Change data types to integer for price_sf and year
    neighborhood_data = neighborhood_data.astype({'PRICE_median': 'int'})
    neighborhood_data = neighborhood_data.astype({'AREA_mean': 'int'})
    neighborhood_data = neighborhood_data.astype({'CENSUS_TRACT': 'int'})

In [35]:
neighborhood_data

,CENSUS_TRACT,PRICE_count,PRICE_median,AREA_mean,ASSESSMENT_NBHD_<lambda>
0,0,11961,0,0,0
1,100,1166,842250,3656,Georgetown
2,202,909,827500,3351,Georgetown
3,300,1182,670000,2825,Glover Park
4,400,297,760000,5373,Observatory Circle
...,...,...,...,...,...
172,10700,260,369950,1875,Central-tri 1
173,10800,224,198000,1130,Foggy Bottom
174,10900,83,75000,2139,Congress Heights
175,11000,577,238000,1906,Southwest Waterfront


In [25]:
    # Read in shapefile and examine data
    dc =  gpd.read_file('../data/Census_Tracts_in_2010.shp')
    # Set the Coordinate Referance System (crs) for projections
    # ESPG code 4326 is also referred to as WGS84 lat-long projection
    dc.crs = {'init': 'epsg:4326'}

    # Rename columns in geojson map file
    dc = dc.rename(columns={'geometry': 'geometry'}).set_geometry('geometry')


    dc.sort_values(by=['TRACT'])

    neighborhood_data.index = neighborhood_data.index.astype(int)
    dc.TRACT = dc.TRACT.astype(int)

In [39]:
neighborhood_data.loc[:,'ASSESSMENT_NBHD_<lambda>']

0                         0
1                Georgetown
2                Georgetown
3               Glover Park
4        Observatory Circle
               ...         
172           Central-tri 1
173            Foggy Bottom
174        Congress Heights
175    Southwest Waterfront
176               Woodridge
Name: ASSESSMENT_NBHD_<lambda>, Length: 177, dtype: object

In [40]:
merged = pd.merge(dc, neighborhood_data, left_on='TRACT', right_on='CENSUS_TRACT', how='left')
merged = merged.dropna()
# Fill the null values
values = {'PRICE_count': 0, 'PRICE_median': 0, 'ASSESSMENT_NBHD_<lambda>': "", 'AREA_mean': 0}
merged = merged.fillna(value=values)

In [41]:
merged

,OBJECTID,TRACT,GEOID,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,...,FAGI_TOT_4,FAGI_MED_4,FAGI_TOT_5,FAGI_MED_5,geometry,CENSUS_TRACT,PRICE_count,PRICE_median,AREA_mean,ASSESSMENT_NBHD_<lambda>
0,1,3400,11001003400,4347,4152,576,3392,20,97,3,...,79636907.0,42596.0,9.209657e+07,46152.0,"POLYGON ((-77.01727 38.92802, -77.01717 38.927...",3400.0,429.0,450000.0,2970.0,Ledroit Park
1,2,3500,11001003500,3740,3594,626,2681,14,106,0,...,78873551.0,40450.0,9.754957e+07,44843.0,"POLYGON ((-77.02205 38.92089, -77.02193 38.920...",3500.0,520.0,430500.0,2275.0,Columbia Heights
2,3,3600,11001003600,4208,4048,1773,1645,13,161,0,...,143081526.0,41616.0,1.780404e+08,46806.0,"POLYGON ((-77.02705 38.92676, -77.02705 38.926...",3600.0,625.0,450000.0,2877.0,Columbia Heights
3,4,3700,11001003700,5763,5507,1915,2527,22,212,10,...,140501730.0,39131.0,1.816520e+08,45690.5,"POLYGON ((-77.03242 38.92657, -77.03241 38.926...",3700.0,891.0,460000.0,1988.0,Columbia Heights
4,5,3800,11001003800,4614,4419,2884,825,22,261,1,...,213544242.0,56380.0,2.356193e+08,60561.0,"POLYGON ((-77.04155 38.91684, -77.04166 38.916...",3800.0,956.0,529950.0,2189.0,Mt. Pleasant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,175,9507,11001009507,1489,1460,53,1304,11,1,0,...,39838674.0,38723.0,4.183457e+07,39851.0,"POLYGON ((-76.99184 38.95741, -76.99164 38.957...",9507.0,238.0,289000.0,2261.0,Riggs Park
175,176,9508,11001009508,3290,3240,257,2821,11,30,0,...,76756174.0,38133.5,8.217265e+07,38670.5,"POLYGON ((-76.99438 38.95728, -76.99419 38.957...",9508.0,331.0,292000.0,2436.0,Riggs Park
176,177,9509,11001009509,2940,2888,165,2592,5,33,0,...,90266682.0,48269.0,9.445337e+07,50505.0,"POLYGON ((-76.98926 38.95242, -76.98927 38.952...",9509.0,520.0,306300.0,2216.0,Brookland
177,178,9601,11001009601,2445,2420,27,2359,3,5,3,...,33222152.0,25221.0,3.456586e+07,27655.0,"POLYGON ((-76.93485 38.90853, -76.93653 38.907...",9601.0,179.0,225000.0,2785.0,Lily Ponds
